# Customizing visual appearance

A HoloViews element, such as those illustrated in the [Introduction](1-Introduction.ipynb)  section, packages your data together with crucial bits of semantic metadata that are required before the data can be interpreted as a visualizable object like a curve or an image.  Even though the resulting object can easily be visualized, it is designed not to contain any information that is solely about visualization -- it only stores declarations about what the data *is*. In turn, the visualization details can be controlled separately, which allows easy customization without complicating the actual data objects.

## Visualizing neural spike trains

We will now introduce the HoloViews options system using another dataset, namely a recording of the electrical spiking activity of a [neuron](https://en.wikipedia.org/wiki/Neuron). We will be visualizing the first trial of this [publicly accessible neural recording](http://www.neuralsignal.org/data/04/nsa2004.4/433l019). We will now import pandas and holoviews and load our data:

In [ ]:
import pandas as pd
import holoviews as hv
spike_train = pd.read_csv('../datasets/spike_train.csv.')
spike_train.head(n=3)

This dataset contains the spike times (in milliseconds) and the average spiking frequency (in Hertz, within a 200 millisecond window) for a total of six seconds (6000 miliseconds). We will now declare ``Curve`` and ``Spike`` elements using this data and combine them into a ``Layout``:

In [ ]:
curve = hv.Curve(spike_train, kdims=['milliseconds'], vdims=['hertz'])
spikes = hv.Spikes(spike_train.sample(300), kdims=['milliseconds'], vdims=[])
curve + spikes

We are now given a purely textual representation of our object as we have no yet loaded the HoloViews extension to enable the plotting system:

In [ ]:
hv.extension('bokeh', 'matplotlib') # Enable the use of both Bokeh and Matplotlib

As the text representation is less useful that the rich representation, this line is normally executed right after importing HoloViews. Note that in the [Introduction](1-Introduction.ipynb), only the 'bokeh' backend was loaded but here we will allow HoloViews to make use of both the [Bokeh](http://bokeh.pydata.org/) and [matplotlib](http://matplotlib.org/) libraries. As 'bokeh' is listed listed first, this is the extension that is now enabled by default.

# Default appearance

With the extension loaded, lets look at the default appearance as rendered with Bokeh:

In [ ]:
curve + spikes

This representation is far more useable than the textual representation even if many aesthetic aspects of it are not to our liking. The ``Curve`` has a useable default representation and there is no need to customize its appearance if we only want a quick overview of the overall trend. The ``Spikes`` representation is *not* useable by default as the plot is nearly solid black even though we already downsampled from 700 spikes to 200 spikes when we declared the element!

This highlights the importance of being able to quickly and easily customize the display of your elements. Sometimes these changes are a matter of personal preference and sometimes these are necessary changes to make the data interpretable as the HoloViews default settings cannot be appropriate for all possible data inputs.

## Customization

Now lets see what we can achieve when we do decide to customize the appearance propely:

In [ ]:
%%output size=150
%%opts Spikes [height=100 width=600 yaxis=None] (color='grey' line_width=0.25)
%%opts Curve [width=600 height=100 xaxis=None show_grid=False tools=['hover']]
%%opts Curve (color='red' line_width=1.5)
curve = hv.Curve(spike_train, kdims=['milliseconds'], vdims=['hertz'])
spikes = hv.Spikes(spike_train, kdims=['milliseconds'], vdims=[])
(curve+spikes).cols(1)

Much better!

A detailed breakdown of this exact customization is given in the [User Guide] but we can use this example to convey a number of important concepts.

* The option system is based around keyword settings.
* You can customize the output format using the ``%%output`` and the element appearance with the ``%%opts`` *cell magics*.
* These *cell magics* affect the display output of the cell they are in, consult the User Guide for equivalent, Python compatible syntax.
* The layout container has a ``cols`` method to specify the number of columns in the layout.

While the ``%%output`` cell magic accepts a simple list of keywords, we see some special syntax used in the ``%%opts`` magic:

* The element type is specified following by special groups of keywords.
* The keywords in square brackets ``[...]`` are ***plot options*** which instruct HoloViews how to build the plot.
* The keywords in parentheses ``(...)`` are **style options** with keywords that are passed directly to the plotting library.

The corresponding [User Guide] entry explains the keywords used in detail but it is possible to state quick simply what we have done: we have elongated the ``Curve`` and ``Scatter`` elements and toggle various axes with the ***plot options***.. We have also specified the color and line widths of the  [Bokeh glyphs](http://bokeh.pydata.org/en/latest/docs/user_guide/plotting.html) with the ***style options***.

As you can see, these tools allow significant customization of how our elements appear. HoloViews offers many other tools for setting options either locally or globally, including the ``%output`` and ``%opts`` *line magics*, the ``.opts`` method on all HoloViews objects and the ``hv.output`` and ``hv.opts`` utilities. All these tools, how they work and details of the opts syntax can be found in the [User Guide].



# Switching to matplotlib

In [ ]:
# HIDDEN CELL!
%output backend='matplotlib'
# New style should follow this by default
%opts Spikes [bgcolor='white']
%opts Curve [bgcolor='white']

Now let's switch our backend to [matplotlib](http://matplotlib.org/) to show the same elements as rendered with different customizations, in a different output format (SVG) with a completely different plotting library:

In [ ]:
%%output size=200 backend='matplotlib' fig='svg'
%%opts Layout [sublabel_format='' vspace=0.1]
%%opts Spikes [aspect=6 yaxis='bare'] (color='red' linewidth=0.25 )
%%opts Curve [aspect=6 xaxis=None show_grid=False] (color='blue' linewidth=0.5 marker='s' ms=2)
(hv.Curve(spike_train, kdims=['milliseconds'], vdims=['hertz'])
 + hv.Spikes(spike_train, kdims=['milliseconds'], vdims=[])).cols(1)

Here we use the same tools with a different plotting extension. Naturally a few changes need to be made:

* A few of the plotting options are different. For instance, ``aspect`` is used instead of setting ``width`` and ``height``.
* The bokeh hover tool is no longer supported as you might expect.
* Some style options have different names, for instance the bokeh ``line_width`` option is called ``linewidth`` in matplotlib.
* Containers such as Layouts have plot option but no style options. Here is adjust the gap betwen the plots using ``vspace``.

Note that you can even write options that work across multiple backends as HoloViews will ignore keywords that are not applicable to the current backend. See the [User Guide] for more details.

## Persistent styles

Let's switch back to the bokeh plotting extension using the ``%output`` *line magic* and try the select operation introduced in the Introduction on the ``spikes`` handled we made earlier:

In [ ]:
%output backend='bokeh' size=150
spikes.select(milliseconds=(2000,4000))

Note how HoloViews remembered the bokeh specific styles we applied to this object earlier! This allows us to style objects once and then keep that styling as we work. You can learn more about the output line magic and the exact semantics of the opts magic in the [User Guide].

## Setting axis labels

If you look closely, the example above might worry you. First we defined our ``Spikes`` element with  ``kdims=['milliseconds']`` which we then used as a keyword argument in ``select`` above. This is also the string used as the axis label. Does this mean we cannot choose any arbitrary axis labels if we want to use the corresponding dimension with ``select``?

Dimensions specified as string are often convenient but behind the scenes, HoloViews always uses a much richer ``Dimensions`` object which you can pass to the ``kdims`` and ``vdims`` explicitly (see the Dimensions User Guide for more information). One of the things each ``Dimension`` object supports is a long, descriptive ``label``  which complements the short programmer-friendly name.

We can set the dimension labels on our existing ``spikes`` object as follows:

In [ ]:
spikes= spikes.redim.label(milliseconds='Time in milliseconds (10⁻³ seconds)')
curve  = curve.redim.label(hertz='Frequency (Hz)')
(curve + spikes).select(milliseconds=(2000,4000)).cols(1)

As you can see we can set long descriptive labels on our dimensions (including unicode) while still making use of the short dimension name in methods such as ``select``.